In [1]:
import ipywidgets as widgets

import rospy

import actionlib
from actionlib import GoalStatus
from assignment2_rt.msg import PlanningAction, PlanningGoal
from assignment2_rt.msg import Robot_info
from nav_msgs.msg import Odometry

In [2]:
pub_position_vel = None
client = None
out_server = widgets.Output()
out = widgets.Output()
ui, x_widget, y_widget = None, None, None

In [3]:
def odom_callback(msg): 
    global pub_position_vel
    robot_info = Robot_info()
    robot_info.x = msg.pose.pose.position.x
    robot_info.y = msg.pose.pose.position.y
    robot_info.vel_x = msg.twist.twist.linear.x
    robot_info.vel_z = msg.twist.twist.angular.z

    pub_position_vel.publish(robot_info)

In [4]:
display(out_server)

Output()

In [5]:
def feedback_callback(feedback):
    global out
    with out:
        if (feedback.stat == "Target reached!"):
            print("Target reached\n{}\nStatus: {}\n".format(feedback.actual_pose, feedback.stat))

def send_goal(client, x, y):
    global out
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    
    goal.target_pose.pose.position.z = 0.0      
    goal.target_pose.pose.orientation.x = 0.0  
    goal.target_pose.pose.orientation.y = 0.0  
    goal.target_pose.pose.orientation.z = 0.0   
    goal.target_pose.pose.orientation.w = 0.0   

    client.send_goal(goal, feedback_cb=feedback_callback)
    with out:
        print("Goal sent")

def cancel_goal(client):
    global out
    with out:
        if client.get_state() in [GoalStatus.ACTIVE, GoalStatus.PENDING]:
            print("Cancelling current goal")
            client.cancel_goal()
            rospy.sleep(0.5)  # Timeout for forward the cancel
            state = client.get_state()
            if state in [GoalStatus.PREEMPTED, GoalStatus.RECALLED]:
                print("Goal successfully cancelled")
            else:
                print("Failed to cancel the goal")
        else:
            print("No active goal to cancel.")

def main():
    global pub_position_vel, client, out_server
    rospy.init_node('action_client')
    pub_position_vel = rospy.Publisher('/robot_information', Robot_info, queue_size=10)
    with out_server:
        print("Wait for the message of topic `/odom`")
    rospy.wait_for_message('/odom', Odometry)
    rospy.Subscriber('/odom', Odometry, odom_callback)

    client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
    with out_server:
        print("Wait the action server starts")
    client.wait_for_server()
    with out_server:
        print("All components started")

try:
    main()
except rospy.ROSInterruptException:
    global out_server
    with out_server:
        print("Program interrupted")

In [6]:
def confirm_clicked(b):
    global x_widget, y_widget, out
    
    if out is not None:
        out.clear_output()
            
    with out:
        x, y = x_widget.value, y_widget.value
        if x is None or y is None:
            print("Enter only valid values for X and Y")
            return
        send_goal(client, x, y)
        print(f"Goal set to (X: {x:.2f}, Y: {y:.2f})")
        
def ask_input():
    global x_widget, y_widget, ui
    
    x_widget = widgets.BoundedFloatText(value=0.0, min=-20.0, max=20.0, step=1, description='X:')
    y_widget = widgets.BoundedFloatText(value=0.0, min=-20.0, max=20.0, step=1, description='Y:')
    
    confirm_button = widgets.Button(description='Send goal')   

    ui = widgets.VBox([x_widget, y_widget, confirm_button])
    display(ui)
    
    confirm_button.on_click(confirm_clicked)

In [7]:
command = widgets.Dropdown(
    options=[('Select a command', 'n'),
            ('Set a new goal (s)', 's'),
             ('Cancel the current goal (c)', 'c'),
             ('Quit the action client (q)', 'q')],
    value='n',
    description='Command:',
)
display(command)

def command_change(change):
    global client
    cmd = change['new']
    if cmd == 'n':
        if ui is not None:
            ui.layout.display = 'none'
    elif cmd == 's':
        if out is not None:
            out.clear_output()
        ask_input()
    elif cmd == 'c':
        if out is not None:
            out.clear_output()
        if ui is not None:
            ui.layout.display = 'none'
        cancel_goal(client)
    elif cmd == 'q':
        command.disabled = True
        with out:
            print("Quit from widget")
        rospy.signal_shutdown("Quit from widget")
        
command.observe(command_change, names='value')

Dropdown(description='Command:', options=(('Select a command', 'n'), ('Set a new goal (s)', 's'), ('Cancel the…

In [8]:
display(out)

Output()